In [1]:
import os

In [2]:
%pwd 

'f:\\Chest_Cancer_Classification_using_MLFlow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Chest_Cancer_Classification_using_MLFlow-and-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import *

In [7]:
class ConfigurationManager:
    def __init__(
        self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        #get file from the url 
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from: {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(f"{prefix}{file_id}", zip_download_dir, quiet=False)
            logger.info(f"File downloaded: {zip_download_dir} of size: {get_size(Path(zip_download_dir))}")
        except Exception as e:
            raise e

    def extract_zip_file(self):
        """Extracts the zip file to the specified directory.
            function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            logger.info(f"Extracting file: {self.config.local_data_file} to {unzip_path}")
            zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed. Files are available at: {unzip_path}")

In [10]:
%pwd

'f:\\Chest_Cancer_Classification_using_MLFlow-and-DVC'

In [11]:
try:  
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-08-15 18:25:27,238: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-15 18:25:27,238: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-15 18:25:27,238: INFO: common: created directory at: artifacts]
[2025-08-15 18:25:27,238: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-15 18:25:27,253: INFO: 3644245465: Downloading file from: https://drive.google.com/file/d/1bkHBKwieJ6P0aMpVYEbbyY9w_sstB4IF/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1bkHBKwieJ6P0aMpVYEbbyY9w_sstB4IF
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1bkHBKwieJ6P0aMpVYEbbyY9w_sstB4IF&confirm=t&uuid=c00bdd16-fefe-4ab6-8de3-401feb291b1f
To: f:\Chest_Cancer_Classification_using_MLFlow-and-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:38<00:00, 3.20MB/s] 


[2025-08-15 18:26:08,915: INFO: 3644245465: File downloaded: artifacts/data_ingestion/data.zip of size: ~ 121464 KB]
[2025-08-15 18:26:08,954: INFO: 3644245465: Extracting file: artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[2025-08-15 18:26:10,515: INFO: 3644245465: Extraction completed. Files are available at: artifacts/data_ingestion]
